# LLM reasoning pop quiz

Do open-sourced LLMs have the reasoning prowess of their closed-sourced siblings?


## Table of Contents

1. Setup
2. Read yaml config
3. Load model
4. Run the quiz

## Setup

In [1]:
# detailed information on the GPU

#!nvidia-smi

In [2]:
#!git clone https://github.com/daniel-furman/LLM-reasoning-pop-quiz.git

In [3]:
#!pip list

In [4]:
# import libraries

import openai
import time
import yaml

# import helpers

from boilers import openai_llm_boiler

/Users/danielfurman/Desktop/github_repos/llm-reasoning-pop-quiz/.venv_quiz/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read in the yaml config for the run

In [5]:
with open("../configs/pop_quiz.yml", "r") as file:
    pop_quiz = yaml.safe_load(file)
pop_quiz

{'prompts': {'zero_shot': ["A juggler has 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?\nLet's respond succinctly and work this out in a step by step way to be sure we have the right answer.",
   "Daniel is in need of a haircut. His barber works Mondays, Wednesdays, and Fridays. So, Daniel went in for a haircut on Sunday. Does this make logical sense?\nLet's respond succinctly and work this out in a step by step way to be sure we have the right answer.",
   "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?\nLet's respond succinctly and work this out in a step by step way to be sure we have the right answer.",
   "Roger has 3 children. Each of his kids invited 4 of their friends to come to the birthday party. All of the friends came to the party. How many children are at the party?\nLet's respond succinctly and work this out in a step by step way to be sure we

## Load the model

In [6]:
# load model
# this cell will take a long time, to avoid: deploy the LLM as an API inference endpoint

model_id = "openai/gpt_3_5_turbo"

model = openai_llm_boiler(model_id)

Load function recognized for openai/gpt_3_5_turbo: gpt_3_5_turbo_loader
Run function recognized for openai/gpt_3_5_turbo: gpt_3_5_turbo


In [7]:
print(model.name, "\n")
print(model.model_id, "\n")

gpt_3_5_turbo 

openai/gpt_3_5_turbo 



In [8]:
print(model.client, "\n")

## Run the model

* For text generation options, refer to [https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextGenerationPipeline](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextGenerationPipeline)
* Below prompts are borrowed from [https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md](https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md)

### Example 1: Zero-shot reasoning conditioned on good performance
* From https://arxiv.org/abs/2205.11916

In [9]:
# run zero shot questions

for itr, prompt in enumerate(pop_quiz["prompts"]["zero_shot"]):
    print(f"Question 1.{itr+1}")
    start_time = time.time()
    generated_text = model.run(
        prompt=prompt,
        temperature=0.01,
        debug=True,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

Question 1.1
*** Prompt
You are a helpful assistant. A juggler has 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?
Let's respond succinctly and work this out in a step by step way to be sure we have the right answer.
*** Generated
To find the number of blue golf balls, we need to determine half of the total number of balls and then half of that result. 

Step 1: Half of 16 balls is 16/2 = 8 balls.
Step 2: Half of the golf balls is 8/2 = 4 balls.

Therefore, there are 4 blue golf balls.
--- 3.690162181854248 seconds ---


Question 1.2
*** Prompt
You are a helpful assistant. Daniel is in need of a haircut. His barber works Mondays, Wednesdays, and Fridays. So, Daniel went in for a haircut on Sunday. Does this make logical sense?
Let's respond succinctly and work this out in a step by step way to be sure we have the right answer.
*** Generated
No, it does not make logical sense for Daniel to go in for a haircut on Sunday 

### Example 2: Least to most prompting
* From https://arxiv.org/abs/2205.10625


In [10]:
# run least to most questions

for itr, prompts in enumerate(pop_quiz["prompts"]["least_to_most"]):
    sub_question_1 = prompts[0]
    print(f"Question 2.{itr+1}")
    # Start with sub question #1

    start_time = time.time()
    res_1 = model.run(
        prompt=sub_question_1,
        temperature=0.01,
        debug=True,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

    # Now do sub question #2 by appending answer to sub question #1
    sub_question_2 = f"{sub_question_1}\n{res_1}\n{prompts[1]}"

    start_time = time.time()
    res_2 = model.run(
        prompt=sub_question_2,
        temperature=0.01,
        debug=True,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

Question 2.1
*** Prompt
You are a helpful assistant. It takes Amy 4 minutes to climb to the top of a slide. It takes her 1 minute to slide down. How long does each trip take?
Let's respond succinctly and work this out in a step by step way to be sure we have the right answer.
*** Generated
To find out how long each trip takes, we need to add the time it takes Amy to climb up the slide and the time it takes her to slide down.

Climbing up the slide takes Amy 4 minutes.
Sliding down the slide takes Amy 1 minute.

So, each trip takes Amy a total of 4 + 1 = 5 minutes.
--- 2.8328440189361572 seconds ---


*** Prompt
You are a helpful assistant. It takes Amy 4 minutes to climb to the top of a slide. It takes her 1 minute to slide down. How long does each trip take?
Let's respond succinctly and work this out in a step by step way to be sure we have the right answer.
To find out how long each trip takes, we need to add the time it takes Amy to climb up the slide and the time it takes her to sl

### Example 3: Tab-CoT

* See https://arxiv.org/abs/2305.17812

In [11]:
# run tab-cot questions

for itr, prompts in enumerate(pop_quiz["prompts"]["tab_cot"]):
    print(f"Question 3.{itr+1}")
    # Start with sub question #1
    sub_question_1 = prompts[0]

    start_time = time.time()
    res_1 = model.run(
        prompt=sub_question_1,
        temperature=0.01,
        debug=True,
    )
    print("--- %s seconds ---" % (time.time() - start_time))
    print("\n")

Question 3.1
*** Prompt
You are a helpful assistant. Jackson is planting tulips. He can fit 6 red tulips in a row and 8 blue tulips in a row. If Jackson buys 36 red tulips and 24 blue tulips, how many rows of flowers will he plant?
Respond as succinctly as possible. Format the response as a completion of this table.
|step|subquestion|procedure|result|
|:---|:----------|:--------|:-----:|
*** Generated
|step|subquestion|procedure|result|
|:---|:----------|:--------|:-----:|
|1|How many rows of red tulips can Jackson plant?|Divide the number of red tulips by the number of red tulips in a row.|6 rows|
|2|How many rows of blue tulips can Jackson plant?|Divide the number of blue tulips by the number of blue tulips in a row.|3 rows|
|3|How many total rows of flowers will Jackson plant?|Add the number of rows of red tulips and the number of rows of blue tulips.|9 rows|
--- 3.5011351108551025 seconds ---


Question 3.2
*** Prompt
You are a helpful assistant. The bakers at the Beverly Hills Bak